In [1]:
import numpy as np
import pandas as pd
import loglikelihood as myfile

In [2]:
ind = range(1,31)
x1 = [0] + [8 if 6<=i and i<=15 else 4 for i in ind]
x2 = [0] + [2.9 if 6<=i and i<=20 else 2.2 if 26<=i else 3.2 for i in ind]
x3 = [0] + [95 if i<=5 or (11<=i and i<=15) and i<=20 else 135 if 26<=i else 60 for i in ind]
x4 = [0] + [3000-300*((i-1)%5) for i in ind]
x = np.array([x1,x2,x3,x4]).T
print(x[:2])

[[   0.     0.     0.     0. ]
 [   4.     3.2   95.  3000. ]]


In [18]:
df = pd.read_csv("clean_train.csv")

df["Assortment"] = df["Assortment"].apply(lambda x: [int(i) for i in x[1:-1].split(",")])
df["Probability"] = df["Probability"].apply(lambda x: [float(i) for i in x[1:-1].split(",")])
df.head(2)

,Assortment,Probability
0,"[0, 2]","[0.6171762553809882, 0.382823744619012]"
1,"[0, 22, 27]","[0.49010203287014026, 0.3856208197783525, 0.12..."


In [4]:
y = myfile.y_np(df["Assortment"], df["Probability"])
print(y[:2])

[[0.61717626 0.         0.38282374 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.        ]
 [0.49010203 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.38562082 0.
  0.         0.         0.         0.12427715 0.         0.
  0.        ]]


In [5]:
# all data used as train
beta = myfile.calc_beta(x,y)
beta

array([ 3.26311685e-01,  4.93574566e-01, -2.35577402e-02, -4.80445945e-04])

In [6]:
import biogeme_nice as myfile1

In [7]:
beta_dict = myfile1.get_beta_dict(beta)
print(beta_dict)

{'B1': 0.3263116854084238, 'B2': 0.49357456645769804, 'B3': -0.02355774017834474, 'B4': -0.00048044594543504777}


In [8]:
# predictions on train data
preds = myfile1.pred_df(beta_dict, x, df["Assortment"])
preds.head(2)

,Assortment,Pred
0,"[0, 2]","[0.6571240323839554, 0.34287596761604455]"
1,"[0, 22, 27]","[0.4321128379943111, 0.514246973950253, 0.0536..."


In [9]:
# show MAE for each row
preds["Truth"] = df["Probability"]
preds["MAE"] = [myfile1.mae(preds["Truth"][i], preds["Pred"][i]) for i in range(len(preds))]
preds

,Assortment,Pred,Truth,MAE
0,"[0, 2]","[0.6571240323839554, 0.34287596761604455]","[0.6171762553809882, 0.382823744619012]",0.039948
1,"[0, 22, 27]","[0.4321128379943111, 0.514246973950253, 0.0536...","[0.49010203287014026, 0.3856208197783525, 0.12...",0.085751
2,"[0, 7, 17, 22]","[0.14281817463394353, 0.5406453819937983, 0.14...","[0.17383055343275639, 0.44812960860117884, 0.2...",0.091340
3,"[0, 17, 7, 21, 14]","[0.11041827420587069, 0.11332050919270566, 0.4...","[0.1338762504510885, 0.1933610050447225, 0.365...",0.047841
4,"[0, 7, 2, 25, 30, 13]","[0.10811334671297132, 0.40926851069254827, 0.0...","[0.09520137647807961, 0.4258881190119289, 0.02...",0.035678
...,...,...,...,...
2495,"[0, 13]","[0.34280834909980323, 0.6571916509001968]","[0.37037705537265336, 0.6296229446273469]",0.027569
2496,"[0, 23, 17]","[0.2940426341859483, 0.40418611218813205, 0.30...","[0.25241095695520893, 0.10976818151988744, 0.6...",0.224033
2497,"[0, 11, 10, 29]","[0.11854048585682368, 0.17033909516641485, 0.6...","[0.13702096921422796, 0.17458857209162085, 0.6...",0.011365
2498,"[0, 16, 25, 13, 1]","[0.16417110927529088, 0.1458706062968337, 0.30...","[0.167781680054737, 0.29015312192773385, 0.148...",0.081898


In [12]:
preds.sort_values("MAE", ascending=False).head(10)

,Assortment,Pred,Truth,MAE
191,"[0, 20, 23]","[0.2527780784811816, 0.3997573809750656, 0.347...","[0.14189009103509673, 0.8207708964274361, 0.03...",0.280676
1086,"[0, 20, 23]","[0.2527780784811816, 0.3997573809750656, 0.347...","[0.14189009103509673, 0.8207708964274361, 0.03...",0.280676
1426,"[0, 21, 20]","[0.2768710268591276, 0.2852696570557488, 0.437...","[0.15204222316960048, 0.009869654687396121, 0....",0.266819
276,"[0, 20, 21]","[0.2768710268591276, 0.43785931608512363, 0.28...","[0.15204222316960048, 0.8380881221430034, 0.00...",0.266819
2236,"[0, 19, 24]","[0.25272456253085435, 0.3460254399689922, 0.40...","[0.16751296154674614, 0.7314159240885767, 0.10...",0.256927
886,"[0, 24, 19]","[0.25272456253085435, 0.40124999750015344, 0.3...","[0.16751296154674614, 0.10107111436467739, 0.7...",0.256927
1976,"[0, 24, 19]","[0.25272456253085435, 0.40124999750015344, 0.3...","[0.16751296154674614, 0.10107111436467739, 0.7...",0.256927
46,"[0, 24, 19]","[0.25272456253085435, 0.40124999750015344, 0.3...","[0.16751296154674614, 0.10107111436467739, 0.7...",0.256927
2056,"[0, 19, 23]","[0.26711089322873754, 0.36572291756841113, 0.3...","[0.1803663835402363, 0.7502619514112845, 0.069...",0.256359
2041,"[0, 22, 19]","[0.2809576123384376, 0.3343608179380959, 0.384...","[0.19057901459639814, 0.04250368080738384, 0.7...",0.254824


In [10]:
# get mean MAE of each row
# rows of different sized assortment contribute equally, assumed to be equally difficult to predict
myfile1.sparse_mae(df["Probability"], preds["Pred"])

0.07447838945958508

In [26]:
# MAE of betas trained on first 80% and tested on last 20%
split = int(0.8*len(df))
beta = myfile.calc_beta(x,y[:split])
beta_dict = myfile1.get_beta_dict(beta)
test = df[split:].reset_index()
test["Pred"] = myfile1.pred_df(beta_dict, x, test["Assortment"])["Pred"]

myfile1.sparse_mae(test["Probability"], test["Pred"])

0.07364777317594033